In [11]:
# Import necessary libraries

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.model_selection import train_test_split, KFold, cross_val_score, ParameterGrid
from sklearn.multioutput import MultiOutputClassifier
from sklearn import tree
from sklearn.metrics import balanced_accuracy_score, classification_report

In [12]:
# Load dataset
dataset= pd.read_csv('dataset.csv')

# Replace 'Unknown' with NaN in Target
dataset['Genetic Disorder'].replace('Unknown', np.nan, inplace=True)
dataset['Disorder Subclass'].replace('Unknown', np.nan, inplace=True)

dataset.dropna(inplace=True)


# Removing non important features
selected_columns = [
    "White Blood cell count (thousand per microliter)",
    "Mother's age",
    "Father's age",
    "Patient Age",
    "Genetic Disorder",
    "Disorder Subclass"
]

# Create a new DataFrame with only the selected columns
dataset = dataset[selected_columns]


# Alocate features and labels

X = dataset.iloc[:, :-2]  # Features
y = dataset.iloc[:, -2:]  # Labels (last two columns)

# Perform one-hot encoding on the categorical features
X = pd.get_dummies(X, drop_first=True)
# drop_first is used to drop one of the columns for each categorical feature to avoid multicollinearity.

# Check if the target variable has more than 2 classes
if y.nunique().any() > 2:
    y = pd.get_dummies(y, drop_first=True)



# Split the data into validation, validation and test
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75,  random_state=1) # training as 75% 
    

In [13]:
# Select and train model of classifier

# Best Hyperparameters: {'estimator__criterion': 'gini', 'estimator__max_depth': 10, 'estimator__max_features': None, 'estimator__min_samples_leaf': 1, 'estimator__min_samples_split': 10, 'estimator__splitter': 'random'}
# Best Mean Accuracy: 0.2690801625415589

dt = MultiOutputClassifier(tree.DecisionTreeClassifier(criterion= 'gini', max_depth= 10, max_features= None, min_samples_leaf= 1, min_samples_split= 10, splitter = 'random', random_state = 1))

kf = KFold(n_splits=5, shuffle=True, random_state=1)

cv_scores = cross_val_score(dt, X_train, y_train, cv=kf)

print("Cross-validated scores:", cv_scores)
print("Mean cross-validated score:", cv_scores.mean())

dt.fit(X_train, y_train)

y_pred = dt.predict(X_test) 


Cross-validated scores: [0.24492058 0.2279276  0.22940525 0.23051348 0.2308829 ]
Mean cross-validated score: 0.2327299593646103


In [14]:
# Separate the multi output targets
y_test_last1 = y_test.iloc[:, -1]
y_test_last2 = y_test.iloc[:, -2]

y_pred_last1 = y_pred[:, -1]
y_pred_last2 = y_pred[:, -2]

# Evaluate the model

balanced_acc_last1 = balanced_accuracy_score(y_test_last1, y_pred_last1)
balanced_acc_last2 = balanced_accuracy_score(y_test_last2, y_pred_last2)


# Use dataset without Unknown values

In [15]:
# Train models without unknown data

# Replace 'Unknown' with NaN
dataset.replace('Unknown', np.nan, inplace=True)

dataset.dropna(inplace=True)

In [16]:
# Alocate features and labels

X = dataset.iloc[:, :-2]  # Features
y = dataset.iloc[:, -2:]  # Labels (last two columns)

# Perform one-hot encoding on the categorical features
X = pd.get_dummies(X, drop_first=True)
# drop_first is used to drop one of the columns for each categorical feature to avoid multicollinearity.

# Check if the target variable has more than 2 classes
if y.nunique().any() > 2:
    y = pd.get_dummies(y, drop_first=True)

# split the data into validation, validation and test

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75,  random_state=1) # training as 75%

In [17]:
# Select and train model of classifier

# Best Hyperparameters: {'estimator__criterion': 'gini', 'estimator__max_depth': 10, 'estimator__max_features': None, 'estimator__min_samples_leaf': 1, 'estimator__min_samples_split': 10, 'estimator__splitter': 'random'}
# Best Mean Accuracy: 0.2690801625415589

dt = MultiOutputClassifier(tree.DecisionTreeClassifier(criterion= 'gini', max_depth= 10, max_features= None, min_samples_leaf= 1, min_samples_split= 10, splitter = 'random', random_state = 1))

kf = KFold(n_splits=5, shuffle=True, random_state=1)  

cv_scores = cross_val_score(dt, X_train, y_train, cv=kf)

print("Cross-validated scores:", cv_scores)
print("Mean cross-validated score:", cv_scores.mean())

dt.fit(X_train, y_train)

y_pred = dt.predict(X_test) 


Cross-validated scores: [0.21042084 0.23313293 0.20374081 0.20440882 0.21910488]
Mean cross-validated score: 0.2141616566466266


In [18]:
# Separate the multi output targets
y_test_last1_new = y_test.iloc[:, -1]
y_test_last2_new = y_test.iloc[:, -2]

y_pred_last1_new = y_pred[:, -1]
y_pred_last2_new = y_pred[:, -2]

# Evaluate the model

balanced_acc_last1_new = balanced_accuracy_score(y_test_last2_new, y_pred_last2_new)
balanced_acc_last2_new = balanced_accuracy_score(y_test_last2_new, y_pred_last2_new)


In [19]:
# Metrics and information for each output column
print('\nClassification Report (Last Column):')
print(classification_report(y_test_last1, y_pred_last1))
print("balanced accuracy:", balanced_acc_last1)

print('\nClassification Report (Second to Last Column):')
print(classification_report(y_test_last2, y_pred_last2))
print("balanced accuracy:", balanced_acc_last2)

print('\n-------------------------REMOVING UNKNOWNS ---------------------------')

print('\nClassification Report (Last Column):')
print(classification_report(y_test_last1_new, y_pred_last1_new))
print("balanced accuracy:", balanced_acc_last1_new)

print('\nClassification Report (Second to Last Column):')
print(classification_report(y_test_last2_new, y_pred_last2_new))
print("balanced accuracy:", balanced_acc_last2_new)


Classification Report (Last Column):
                                     precision    recall  f1-score   support

                        Alzheimer's       0.00      0.00      0.00        32
                             Cancer       0.00      0.00      0.00        22
                    Cystic fibrosis       0.18      0.02      0.04       745
                           Diabetes       0.13      0.00      0.01       417
                    Hemochromatosis       0.13      0.01      0.01       315
Leber's hereditary optic neuropathy       0.00      0.00      0.00       141
                     Leigh syndrome       0.27      0.87      0.41      1186
             Mitochondrial myopathy       0.24      0.12      0.16      1016
                          Tay-Sachs       0.13      0.00      0.01       638

                           accuracy                           0.26      4512
                          macro avg       0.12      0.11      0.07      4512
                       weighted avg 

c:\Users\Sofia\miniconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Sofia\miniconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Sofia\miniconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Sofia\miniconda3\Lib\site-packages\sklearn\

                                              precision    recall  f1-score   support

 Mitochondrial genetic inheritance disorders       0.52      0.95      0.67      2343
Multifactorial genetic inheritance disorders       0.11      0.00      0.01       471
            Single-gene inheritance diseases       0.37      0.04      0.07      1698

                                    accuracy                           0.51      4512
                                   macro avg       0.33      0.33      0.25      4512
                                weighted avg       0.42      0.51      0.38      4512

balanced accuracy: 0.33228622240814276

-------------------------REMOVING UNKNOWNS ---------------------------

Classification Report (Last Column):
                                     precision    recall  f1-score   support

                        Alzheimer's       0.00      0.00      0.00        24
                             Cancer       0.00      0.00      0.00         9
              

c:\Users\Sofia\miniconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Sofia\miniconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
